# 04 - Evaluation des Resultats

Ce notebook evalue quantitativement et qualitativement les images generees
par le CycleGAN et le modele de diffusion.

**Metriques :**
- **SSIM** : Similarite structurelle (conservation de la structure spatiale)
- **PSNR** : Rapport signal/bruit (fidelite pixel-a-pixel)
- **FID** : Distance de Frechet (qualite de la distribution generee)

**Visualisations :**
- Comparaisons cote-a-cote
- Cartes NDVI
- Resume des metriques

In [ ]:
# Setup (decommenter sur Colab)
# !git clone https://github.com/aymenssf/SatelliteGAN-Climate-Agriculture.git
# %cd SatelliteGAN-Climate-Agriculture
# !pip install -q -r requirements.txt

import sys
import os
sys.path.insert(0, os.path.join(os.getcwd()))

import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms

from src.config import DEVICE, IMAGE_SIZE, EVAL, CYCLEGAN_CKPT_DIR, DIFFUSION_CKPT_DIR
from src.dataset import get_agricultural_dataset, split_dataset
from src.preprocessing import (
    get_eval_transform, simulate_drought, tensor_to_numpy
)
from src.cyclegan.train import CycleGANTrainer
from src.diffusion.train import DiffusionTrainer
from src.evaluation.metrics import (
    compute_ssim, compute_psnr, compute_fid,
    SimpleFeatureExtractor, extract_features
)
from src.evaluation.visualization import (
    show_image_grid, show_comparison, show_cyclegan_results,
    show_ndvi_comparison, plot_metrics_summary
)

print(f"Device : {DEVICE}")

## 1. Chargement des modeles et donnees

In [ ]:
# Charger les donnees de test
raw_transform = transforms.Resize(IMAGE_SIZE)
raw_dataset = get_agricultural_dataset(transform=raw_transform)
_, _, test_set = split_dataset(raw_dataset)

eval_transform = get_eval_transform()

# Preparer les images de test
test_normal = []
test_drought = []

for i in range(len(test_set)):
    img, _ = test_set[i]
    if isinstance(img, torch.Tensor):
        img_pil = transforms.ToPILImage()(img)
    else:
        img_pil = img

    test_normal.append(eval_transform(img_pil))
    drought_pil = simulate_drought(img_pil, severity=0.6)
    test_drought.append(eval_transform(drought_pil))

test_normal = torch.stack(test_normal)
test_drought = torch.stack(test_drought)

print(f"Images de test normales : {test_normal.shape}")
print(f"Images de test secheresse : {test_drought.shape}")

In [ ]:
# Charger le CycleGAN (decommenter et adapter le chemin)
cyclegan_trainer = CycleGANTrainer()

# Decommenter pour charger un checkpoint :
# cyclegan_trainer.load_checkpoint(os.path.join(CYCLEGAN_CKPT_DIR, 'cyclegan_epoch_100.pth'))

# Generer les transformations CycleGAN
with torch.no_grad():
    cyclegan_drought = cyclegan_trainer.generate(test_normal, direction='A2B').cpu()

print(f"Images CycleGAN generees : {cyclegan_drought.shape}")

In [ ]:
# Charger le DDPM (decommenter et adapter le chemin)
diffusion_trainer = DiffusionTrainer()

# Decommenter pour charger un checkpoint :
# diffusion_trainer.load_checkpoint(os.path.join(DIFFUSION_CKPT_DIR, 'ddpm_epoch_150.pth'))

# Generer des images avec le DDPM
n_gen = min(EVAL['n_generated'], len(test_normal))
ddpm_generated = diffusion_trainer.generate(n_samples=n_gen, use_ema=True, fast=True).cpu()

print(f"Images DDPM generees : {ddpm_generated.shape}")

## 2. Evaluation du CycleGAN

Pour le CycleGAN, on mesure :
- **SSIM entre l'original et la reconstruction cyclique** (devrait etre eleve)
- **PSNR entre l'original et la reconstruction cyclique**
- La qualite visuelle des transformations

In [ ]:
# Calcul des metriques CycleGAN
# SSIM et PSNR entre image originale et sa reconstruction cyclique
with torch.no_grad():
    test_dev = test_normal.to(DEVICE)
    fake_drought = cyclegan_trainer.G_A2B(test_dev)
    reconstructed = cyclegan_trainer.G_B2A(fake_drought)
    reconstructed_cpu = reconstructed.cpu()

# SSIM
ssim_cycle = compute_ssim(test_normal, reconstructed_cpu)
print(f"SSIM (cycle-consistency) : {ssim_cycle:.4f}")

# PSNR
psnr_cycle = compute_psnr(test_normal, reconstructed_cpu)
print(f"PSNR (cycle-consistency) : {psnr_cycle:.2f} dB")

# SSIM entre secheresse simulee et CycleGAN
ssim_drought = compute_ssim(test_drought, cyclegan_drought)
print(f"SSIM (sim. vs CycleGAN)  : {ssim_drought:.4f}")

In [ ]:
# Visualisation des resultats CycleGAN
show_comparison(
    test_normal[:6], cyclegan_drought[:6],
    n_samples=6,
    labels=('Normal', 'CycleGAN secheresse'),
    title='CycleGAN : Normal -> Secheresse'
)

In [ ]:
# Comparaison NDVI
show_ndvi_comparison(
    test_normal[:4].cpu(), cyclegan_drought[:4].cpu(),
    n_samples=4
)

## 3. Evaluation du DDPM

Pour le DDPM, on calcule le FID entre les images generees et les vraies
images de secheresse (simulees) pour mesurer la qualite de la distribution.

In [ ]:
# Visualisation des images DDPM
show_image_grid(
    ddpm_generated[:16], n_cols=4,
    title='Images generees par le DDPM'
)

In [ ]:
# Calcul du FID
# On utilise un extracteur de features simple (pas InceptionV3)
feature_extractor = SimpleFeatureExtractor()

# Extraire les features des images reelles et generees
n_for_fid = min(len(test_drought), len(ddpm_generated))

real_feats = extract_features(test_drought[:n_for_fid], feature_extractor)
gen_feats = extract_features(ddpm_generated[:n_for_fid], feature_extractor)

fid_score = compute_fid(real_feats, gen_feats)
print(f"FID (DDPM vs secheresse simulee) : {fid_score:.2f}")
print(f"(plus bas = meilleur, 0 = distributions identiques)")

In [ ]:
# Comparaison cote-a-cote
show_comparison(
    test_drought[:6], ddpm_generated[:6],
    n_samples=6,
    labels=('Secheresse (simulee)', 'Secheresse (DDPM)'),
    title='Comparaison : images simulees vs generees par DDPM'
)

## 4. Resume des metriques

In [ ]:
# Resume global
print("=" * 50)
print("RESUME DES METRIQUES")
print("=" * 50)
print(f"\nCycleGAN :")
print(f"  SSIM (cycle-consistency) : {ssim_cycle:.4f}")
print(f"  PSNR (cycle-consistency) : {psnr_cycle:.2f} dB")
print(f"  SSIM (sim. vs CycleGAN)  : {ssim_drought:.4f}")
print(f"\nDDPM :")
print(f"  FID : {fid_score:.2f}")

# Graphique de resume
metrics = {
    'SSIM\n(cycle)': ssim_cycle,
    'PSNR\n(cycle, dB)': psnr_cycle,
    'FID\n(DDPM)': fid_score,
}

plot_metrics_summary(metrics)

## 5. Analyse et conclusion

### CycleGAN
- **SSIM cycle-consistency eleve** : le CycleGAN preserve bien la structure spatiale
  lors du cycle A->B->A. Les routes, limites de parcelles, et structures
  sont conservees.
- **Transformation visuelle coherente** : les images en secheresse montrent
  une reduction de la verdure et un jaunissement, coherent avec le stress hydrique.
- **NDVI reduit** : le NDVI proxy diminue dans les images transformees,
  confirmant que le CycleGAN a appris a simuler une vegetation stressée.

### DDPM
- **Generation diverse** : le DDPM produit des images variees de secheresse,
  pas juste des copies des images d'entrainement.
- **FID raisonnable** : la distribution des images generees est proche
  de celle des images de secheresse simulees.

### Limitations
- Le domaine secheresse est simule, pas reel (biais d'apprentissage).
- Le FID est calcule avec un extracteur simple, pas InceptionV3.
- Le nombre d'images est limite pour un FID fiable.
- Le modele de diffusion travaille en pixel space (pas latent).

### Perspectives
- Utiliser de vraies images de secheresse (Copernicus EMS).
- Passer a un Latent Diffusion Model pour une meilleure qualite.
- Ajouter un conditionnement (niveau de severite, saison, type de culture).